<a href="https://colab.research.google.com/github/shauravmahmud/Signal-Processing-Practice/blob/main/Sound_Processing_for_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://goo.gl/8hY5ER

--2021-09-25 15:19:15--  https://goo.gl/8hY5ER
Resolving goo.gl (goo.gl)... 74.125.137.113, 74.125.137.101, 74.125.137.102, ...
Connecting to goo.gl (goo.gl)|74.125.137.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://zenodo.org/record/1203745/files/UrbanSound8K.tar.gz [following]
--2021-09-25 15:19:15--  https://zenodo.org/record/1203745/files/UrbanSound8K.tar.gz
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6023741708 (5.6G) [application/octet-stream]
Saving to: ‘8hY5ER’

8hY5ER              100%[===================>]   5.61G  17.9MB/s    in 5m 39s  

2021-09-25 15:24:55 (17.0 MB/s) - ‘8hY5ER’ saved [6023741708/6023741708]



In [2]:
!tar xf 8hY5ER

In [3]:
!ls UrbanSound8K/audio/

fold1  fold10  fold2  fold3  fold4  fold5  fold6  fold7  fold8	fold9


In [4]:
!ls UrbanSound8K/audio/fold1

101415-3-0-2.wav   146186-5-0-8.wav    177621-0-0-145.wav  40722-8-0-5.wav
101415-3-0-3.wav   146186-5-0-9.wav    177621-0-0-147.wav  40722-8-0-6.wav
101415-3-0-8.wav   147764-4-0-0.wav    177621-0-0-27.wav   40722-8-0-7.wav
102106-3-0-0.wav   147764-4-1-0.wav    177621-0-0-2.wav    40722-8-0-8.wav
102305-6-0-0.wav   147764-4-2-0.wav    177621-0-0-31.wav   43786-3-0-0.wav
102842-3-0-1.wav   147764-4-3-0.wav    177621-0-0-35.wav   43786-3-1-0.wav
102842-3-1-0.wav   147764-4-4-0.wav    177621-0-0-46.wav   43806-8-0-0.wav
102842-3-1-5.wav   147764-4-4-1.wav    177621-0-0-47.wav   44736-5-0-0.wav
102842-3-1-6.wav   147764-4-4-2.wav    177621-0-0-48.wav   44736-5-0-1.wav
103074-7-0-0.wav   147764-4-5-0.wav    177621-0-0-54.wav   46654-6-0-0.wav
103074-7-0-1.wav   147764-4-6-0.wav    177621-0-0-57.wav   46656-6-0-0.wav
103074-7-0-2.wav   147764-4-7-0.wav    177621-0-0-80.wav   46656-6-1-0.wav
103074-7-1-0.wav   147764-4-8-0.wav    177621-0-0-88.wav   46656-6-2-0.wav
103074-7-1-1.wav   150341

In [6]:
data_dir = 'UrbanSound8K/audio/fold1'

In [7]:
import IPython
IPython.display.Audio("UrbanSound8K/audio/fold1/99180-9-0-7.wav")

In [25]:
labels  = ['air_conditioner','car_horn','children_playing',
           'dog_bark','drilling','engine_idling','gun_shot','jackhammer','siren','street_music']

In [26]:
!mkdir 'UrbanSound8K/spectrogram'

mkdir: cannot create directory ‘UrbanSound8K/spectrogram’: File exists


In [27]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import specgram
import librosa
import numpy as np

In [28]:
def create_fold_spectrograms(fold):
    spectrogram_path = Path('UrbanSound8K/spectrogram/')  
    audio_path = Path('UrbanSound8K/audio/')  
    print(f'Processing fold {fold}')
    os.mkdir(spectrogram_path/fold)
    for audio_file in list(Path(audio_path/f'fold{fold}').glob('*.wav')):
        samples, sample_rate = librosa.load(audio_file)
        fig = plt.figure(figsize=[0.72,0.72])
        ax = fig.add_subplot(111)
        ax.axes.get_xaxis().set_visible(False)
        ax.axes.get_yaxis().set_visible(False)
        ax.set_frame_on(False)
        filename  = spectrogram_path/fold/Path(audio_file).name.replace('.wav','.png')
        S = librosa.feature.melspectrogram(y=samples, sr=sample_rate)
        librosa.display.specshow(librosa.power_to_db(S, ref=np.max))
        plt.savefig(filename, dpi=400, bbox_inches='tight',pad_inches=0)
        plt.close('all')

In [29]:
!mkdir 'UrbanSound8K/data'

mkdir: cannot create directory ‘UrbanSound8K/data’: File exists


In [30]:
import os
import glob
import shutil
from pathlib import Path  

In [32]:
data_path = Path('UrbanSound8K/data/') 
spectrogram_path = Path('UrbanSound8K/spectrogram/')  
def create_fold_directory(fold):
    png_files =  list(Path(spectrogram_path/fold).glob('*.png'))
    os.mkdir(data_path/fold)
    os.mkdir(data_path/fold/'train')
    os.mkdir(data_path/fold/'valid')
    for label in labels:
        os.mkdir(data_path/fold/'train'/label)
        os.mkdir(data_path/fold/'valid'/label)
     
    for file in png_files:
        label = file.as_posix().split('-')[1]
        shutil.copyfile(file, data_path/fold/'valid'/labels[int(label)]/file.name)   
    for i in range(1, 11):       
        if str(i) == fold:
            continue
        png_files =  list(Path(spectrogram_path/str(i)).glob('*.png'))
        for file in png_files:
            label = file.as_posix().split('-')[1]
            shutil.copyfile(file, data_path/fold/'train'/labels[int(label)]/file.name) 

In [33]:
for i in range(1, 11):
    create_fold_directory(str(i))

In [34]:
!ls UrbanSound8K/data

1  10  2  3  4	5  6  7  8  9


In [35]:
!ls UrbanSound8K/data/1

train  valid


In [43]:
!ls UrbanSound8K/data/1/train

air_conditioner  children_playing  drilling	  gun_shot    siren
car_horn	 dog_bark	   engine_idling  jackhammer  street_music


In [44]:
pwd

'/content'

In [47]:
!ls /content

8hY5ER	sample_data  UrbanSound8K


In [60]:
!ls UrbanSound8K/data/10/train/

air_conditioner  children_playing  drilling	  gun_shot    siren
car_horn	 dog_bark	   engine_idling  jackhammer  street_music
